<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/0604_Deduping_PipelineReady.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Load data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'
df = pd.read_csv(os.path.join(directory, 'combined_metadata_final12_features.csv'))
df=df[['sourcename', 'title']]
df.head()

# 2. Preprocessing titles for NLP

In [ ]:
def remove_non_english(df):
  def is_english(text):
    words = word_tokenize(text)
    english_word_count = sum(len(word) >= 2 for word in words)
    return english_word_count / len(words) >= 0.9

  df_filtered = df[df['title'].apply(is_english)]
  return df_filtered

df['title'] = df['title'].astype(str)
df = df[~df['title'].str.contains('و|д|я|š|ü|à|é|ś|ä|ú|의|θ|λ|ό|τ|η|τ|α|の|と', regex=True)]
df = remove_non_english(df.copy())

In [ ]:
df['sourcename'] = df['sourcename'].apply(lambda x: x.replace('metadata_', ''))
df['sourcename'] = df['sourcename'].apply(lambda x: x.replace('_unifiedColName.csv', ''))
df['sourcename'].replace(['url1_ie_table_complete', 'scopus_fixed_encoding','SSRN'], ['3ie', 'scopus','ssrn'], inplace=True)


# 3. Get word count for grouping
This is an additional featue to run the deduping faster, however it is not a must-step to run.

In [ ]:

def count_words(title):
  words = title.split()
  return len(words)

df['word_count'] = df['title'].apply(count_words)

In [ ]:
min=5
max=20

df = df[(df['word_count'] >= min) & (df['word_count'] <= max)]

title_sample_size= 1000000

def sample_dataframe(df, sample_size):
  if len(df) > sample_size:
    return df.sample(n=sample_size, random_state=321)
  else:
    return df


df_title1 = sample_dataframe(df, title_sample_size)
df_title1 = df_title1.rename(columns={"title": "title1","sourcename": "sourcename1","word_count": "word_count1"})

df_title2 = df.drop(df_title1.index) # OR below line
#df_title2 = df[~df['title'].isin(df_title1['title1'])]
df_title2 = df_title2.rename(columns={"title": "title2","sourcename": "sourcename2","word_count": "word_count2"})
df_title2=df_title2_2

# 4. Make pair of titles for deduping

In [ ]:
df_title1.to_csv(os.path.join(directory, 'df_title1.csv'), index=False)
df_title2.to_csv(os.path.join(directory, 'df_title2.csv'), index=False)
df_title1 = df_title1.reset_index(drop=True)
df_title2 = df_title2.reset_index(drop=True)
df_pair=pd.concat([df_title1, df_title2], axis=1)
df_pair = df_pair.drop(df_pair.loc[df_pair['sourcename1'] == df_pair['sourcename2']].index)
df_pair = df_pair.drop(df_pair.loc[(df_pair['word_count2'] > df_pair['word_count1']+3) | (df_pair['word_count2'] < df_pair['word_count1']-3)].index)
df_pair.to_csv(os.path.join(directory, 'df_pair.csv'), index=False)

In [ ]:
def sample_dataframe(df, sample_size):
  if len(df) > sample_size:
    return df.sample(n=sample_size, random_state=321)
  else:
    return df

title_sample_size=2000
df_pair = sample_dataframe(df_pair, title_sample_size)
df_pair.to_csv(os.path.join(directory, 'df_pair_2k.csv'), index=False)

# 5. Compute distance

In [ ]:
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0]
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n
    return np.array(array)

vects = []
for title in df_pair.title1:
    vects.append(vectorize(title))

In [ ]:
df_pair['vects1']=vects

In [ ]:
vects_2 = []
for title in df_pair.title2:
    vects_2.append(vectorize(title))
df_pair['vects2']=vects_2

In [ ]:
def l1_distance(v1, v2):
  # Ensure both arrays have the same size
  if len(v1) != len(v2):
    raise ValueError("Input vectors must have the same size")
  # Calculate absolute difference between corresponding elements
  diff = np.abs(np.array(v1) - np.array(v2))
  # Return the sum of absolute differences (L1 norm)
  return np.sum(diff)

# Apply the function to each pair of vectors in the columns
df_pair['L1_distance'] = df_pair.apply(lambda row: l1_distance(row['vects1'], row['vects2']), axis=1)
df_pair['annotation_label'] = df_pair.apply(lambda row: 1 if row['title1'] == row['title2'] else 0, axis=1)
df_pair.to_csv(os.path.join(directory, '0503_df_pair_2k_distance_withL1.csv'), index=False)

df_pair will contain list of titles that is examined for deduplication. Any item that is labeld as 1 from 'annotation_label' variable indicates duplicated titles.